# **Step 3:** Forgery Detection Model

In [ ]:
base_dir = "/content/indian-document-forgery-detection/dataset"

# Create the base directory
os.makedirs(base_dir, exist_ok=True)

# Define class names
class_names = ["Real", "Forged", "Mask"]

# Loop through the class names and create subdirectories
for class_name in class_names:
    class_dir = os.path.join(base_dir, class_name)
    os.makedirs(class_dir, exist_ok=True)

In [ ]:

def gen(field_coords,data,last,index,img, image_name):
    length = len(field_coords)
    for i in range(length):

        data[index] = field_coords[i]  # Store the field and coordinates

        if index == last:  # If we have a complete permutation
            new_img = img.copy()
            forged_mask = np.zeros_like(img[:, :, 0])

            forged_name = f"{image_name[:-4]}_{'_'.join([f'{field_name[0]}' for i, field_name in enumerate(data)])}.jpg"


            for i, field_name in enumerate(data):

              for j, (other_field_name, other_coords) in enumerate(field_coords):
                if i == j:
                  dest_coords = other_coords

              for j,(other_field_name, other_coords) in enumerate(field_coords):

                if (field_name[0]==other_field_name and j!=i):
                  forged_mask[dest_coords[0]['y']:dest_coords[0]['y'] + dest_coords[0]['height'],
                            dest_coords[0]['x']:dest_coords[0]['x'] + dest_coords[0]['width']] = 255

                  forged_mask_3_channel = cv2.cvtColor(forged_mask, cv2.COLOR_GRAY2BGR)  # Convert to BGR if needed
                  mask_image_save_path = os.path.join(base_dir, "Mask", forged_name)
                  cv2.imwrite(mask_image_save_path, forged_mask_3_channel)


            for i, field_name in enumerate(data):

              for j,(other_field_name, other_coords) in enumerate(field_coords):
                if i==j:
                  dest_coords=other_coords

              for j,(other_field_name, other_coords) in enumerate(field_coords):

                if (field_name[0]==other_field_name and j!=i):


                  src_region = img[other_coords[0]['y']:other_coords[0]['y'] + other_coords[0]['height'],other_coords[0]['x']:other_coords[0]['x'] + other_coords[0]['width']]

                  src_region_resized = cv2.resize(src_region,
                                                (dest_coords[0]['width'], dest_coords[0]['height']))
                  new_img[dest_coords[0]['y']:dest_coords[0]['y'] + dest_coords[0]['height'],
                  dest_coords[0]['x']:dest_coords[0]['x'] + dest_coords[0]['width']] = src_region_resized
                  forged_image_save_path = os.path.join(base_dir, "Forged", forged_name)

                  cv2.imwrite(forged_image_save_path, new_img)
                  # Upload to GitHub

                  break

                else:
                  continue


        else:
            gen(field_coords, data, last, index + 1, img, image_name)



def generate():
    for i in range(100):
        if(i<10):
            image=f'0{i}.jpg'

        else:
            image=f'{i}.jpg'


        image_Path = repository_path + image
        img = cv2.imread(image_Path)

        real_image_save_path = os.path.join(base_dir, "Real", image)
        shutil.copy(image_Path, real_image_save_path)

        with open(json_path) as f:
          data = json.load(f);

        # Extract metadata paths
        region = [data['_via_img_metadata'][key]['regions']
                  for key in data['_via_img_metadata']]
        reg = region[i]

        birth_date_coords = []
        expiry_date_coords = []
        issue_date_coords = []



        surname_coords = []
        name_coords = []



        nationality_coords = []
        birth_place_coords = []


        number_coords = []
        id_number_coords = []



        for region in reg:
          if region['region_attributes']['field_name'] == 'birth_date':
            birth_date_coords.append(region['shape_attributes'])
          elif region['region_attributes']['field_name'] == 'expiry_date':
            expiry_date_coords.append(region['shape_attributes'])
          elif region['region_attributes']['field_name'] == 'issue_date':
            issue_date_coords.append(region['shape_attributes'])
          elif region['region_attributes']['field_name'] == 'surname':
            surname_coords.append(region['shape_attributes'])
          elif region['region_attributes']['field_name'] == 'name':
            name_coords.append(region['shape_attributes'])
          elif region['region_attributes']['field_name'] == 'nationality':
            nationality_coords.append(region['shape_attributes'])
          elif region['region_attributes']['field_name'] == 'birth_place':
            birth_place_coords.append(region['shape_attributes'])
          elif region['region_attributes']['field_name'] == 'number':
            number_coords.append(region['shape_attributes'])
          elif region['region_attributes']['field_name'] == 'id_number':
            id_number_coords.append(region['shape_attributes'])
        group1=[('birth_date', birth_date_coords), ('expiry_date', expiry_date_coords), ('issue_date', issue_date_coords)]
        group2=[('surname', surname_coords), ('name', name_coords)]
        group3=[('nationality', nationality_coords), ('birth_place', birth_place_coords)]
        group4=[('number', number_coords), ('id_number', id_number_coords)]
        groups = [
            group1,
            group2,
            group3,
            group4
        ]

        # Iterate through each group
        # Iterate through each group
        for group in groups:
            if len(group) < 2:  # Skip groups with less than 2 fields
                continue

            # Get field names from the group
            field_names = [item[0] for item in group]
            all_coords = [item[1] for item in group]
            length=len(group)
            data=[None] * length

            gen(group, data, length-1,0,img,image)


generate()

In [ ]:
## Count the data instants
base_dir = "/content/indian-document-forgery-detection/dataset"
forged_path = "/content/indian-document-forgery-detection/dataset/Forged"
real_path = "/content/indian-document-forgery-detection/dataset/Real"
def count_images_in_category(category_dir):
    return len(os.listdir(category_dir))

##Looping over the classes for training category
# for class_name in class_names:
#     train_category= os.path.join(base_dir, class_name)
count= count_images_in_category(real_path)
print(f"Number of images in {class_name} training category: {count}")
count= count_images_in_category(forged_path)
print(f"Number of images in {class_name} training category: {count}")

In [ ]:
## Define a function to copy a specified number of random samples from source to destination
def copy_random_samples(src_dir, dest_dir, samples):
    files = random.sample(os.listdir(src_dir), samples)
    for file in files:
        src_path = os.path.join(src_dir, file)
        dest_path = os.path.join(dest_dir, file)
        shutil.copy(src_path, dest_path)

In [ ]:
original_path = '/content/indian-document-forgery-detection/dataset'
# Destination paths
train_path = os.path.join(original_path, 'train')
val_path = os.path.join(original_path, 'val')
test_path = os.path.join(original_path, 'test')

# Create directories
for path in [train_path, val_path, test_path]:
    os.makedirs(path, exist_ok=True)

## DEFINE CLASSES AND SAMPLE SIZES ##
## Define classes in the dataset
classes = ["Forged", "Real"]

## Set the number of samples for training, validation, and testing

sample_train = [2450,70]
sample_val = [700,20]
sample_test = [350,10]

In [ ]:
test_path= "/content/indian-document-forgery-detection/dataset/test"
train_path= "/content/indian-document-forgery-detection/dataset/train/im"
val_path= "/content/indian-document-forgery-detection/dataset/val/im"

## FUNCTION TO COPY RANDOM SAMPLES ##

## COPY SAMPLES FOR TRAINING ##
for i, class_name in enumerate(classes):
    src_class_dir = os.path.join(original_path, class_name)
    dest_class_dir = os.path.join(train_path, class_name)
    os.makedirs(dest_class_dir, exist_ok=True)

    # Get the sample number for the current class
    samples1 = sample_train[i]

    # Copy the random samples
    copy_random_samples(src_class_dir, dest_class_dir, samples1)

## COPY SAMPLES FOR VALIDATION ##

## Iterate over classes to copy random samples for validation
for i, class_name in enumerate(classes):
    src_class_dir = os.path.join(original_path, class_name)
    dest_class_dir = os.path.join(val_path, class_name)
    os.makedirs(dest_class_dir, exist_ok=True)


    # Get the sample number for the current class
    samples2 = sample_val[i]

    # Copy the random samples
    copy_random_samples(src_class_dir, dest_class_dir, samples2)


## COPY SAMPLES FOR TESTING ##
## Iterate over classes to copy random samples for testing
for i, class_name in enumerate(classes):
    src_class_dir = os.path.join(original_path, class_name)
    dest_class_dir = os.path.join(test_path, class_name)
    os.makedirs(dest_class_dir, exist_ok=True)
    # Get the sample number for the current class
    samples3 = sample_test[i]

    # Copy the random samples
    copy_random_samples(src_class_dir, dest_class_dir, samples3)



## Defining a function to count the data instants
def count_images_in_category(category_dir):
    return len(os.listdir(category_dir))

##Looping over the classes for training category
for class_name in classes:
    train_category= os.path.join(train_path, class_name)
    count= count_images_in_category(train_category)
    print(f"Number of images in {class_name} training category: {count}")


for class_name in classes:
    test_category= os.path.join(test_path, class_name)
    count = count_images_in_category(test_category)
    print(f"Number of images in {class_name} category (Testing): {count}")



for class_name in classes:
    val_category= os.path.join(val_path, class_name)
    count = count_images_in_category(val_category)
    print(f"Number of images in {class_name} category (Validation): {count}")

In [ ]:
img_height = 256
img_width = 256


batch_size = 1

image_datagen_train = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                         horizontal_flip=True)


image_generator_train = image_datagen_train.flow_from_directory(
    '/content/indian-document-forgery-detection/dataset/train/im',
    class_mode='binary',
    color_mode='rgb',
    target_size=(img_height,img_width),
    batch_size=batch_size,
    seed=2)

image_datagen_valid = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


image_generator_valid = image_datagen_valid.flow_from_directory(
    '/content/indian-document-forgery-detection/dataset/val/im',
    class_mode="binary",
    color_mode='rgb',
    target_size=(img_height,img_width),
    batch_size=batch_size,
    seed=2)

train_generator = image_generator_train
validation_generator = image_generator_valid

In [ ]:
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(img_height,img_width,3),
                   pooling='avg',classes=2,
                   weights='imagenet')
for layer in pretrained_model.layers: # dont learn weights again just use the exisiting one
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(1, activation='sigmoid'))

In [ ]:
resnet_model.summary()

In [ ]:
# optimizer = keras.optimizers.Adam(learning_rate=0.001)
resnet_model.compile(optimizer="adam",loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
epochs=20
# history = model.fit_generator(
#         train_generator,
#         epochs=nb_epochs,
#         validation_data=validation_generator,
#         verbose=1,
#         steps_per_epoch=10//batch_size,
#         validation_steps=10//batch_size,
#         callbacks=cal_b)


history = resnet_model.fit(
  train_generator,
  validation_data=validation_generator,
  epochs=epochs,
  verbose=1,
        steps_per_epoch=20//batch_size,
        validation_steps=10//batch_size,

)

In [ ]:
image_datagen_test = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                         horizontal_flip=True)



image_generator_test = image_datagen_test.flow_from_directory(
    '/content/indian-document-forgery-detection/dataset/test',
    class_mode="binary",
    color_mode='rgb',
    target_size=(img_height,img_width),
    batch_size=batch_size,
    seed=2)




test_generator =image_generator_test

results = resnet_model.evaluate(test_generator)
print("Test Loss:", results[0])
print("Test Accuracy:", results[1])

In [ ]:
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

y_pred = resnet_model.predict(test_generator)
y_pred_classes = (y_pred > 0.5).astype(int)

y_true = test_generator.classes

plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix(y_true, y_pred_classes), annot=True, fmt='d', cmap='Blues',
            xticklabels=["Forged", "Real"],
            yticklabels=["Forged ", "Real"])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
filenames = test_generator.filenames
for i in range(len(filenames)):
    img_path = os.path.join(test_path, filenames[i])
    img = image.load_img(img_path, target_size=(256, 256))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = resnet_model.predict(img_array)
    predicted_class = int(np.round(prediction)[0])

    true_class = int(y_true[i])

    plt.imshow(img)
    plt.title(f"True Class: {true_class}, Predicted Class: {predicted_class}")
    plt.axis('off')
    plt.show()

In [ ]:
#model number 2
model_3 = Sequential([
    Conv2D(32,(3,3) , activation = "relu" , input_shape = (256,256,3),) , # 32 is the number of filters , (3,3) is the size of the filter , input shape is the size of images the models is expecting
    MaxPooling2D((2,2)),

    Conv2D(32,(3,3) , activation = "relu" ),
    MaxPooling2D((2,2)),
    Conv2D(32,(3,3) , activation = "relu" ),
    MaxPooling2D((2,2)),
    Conv2D(32,(3,3) , activation = "relu" ),
    MaxPooling2D((2,2)),
    Conv2D(32,(3,3) , activation = "relu" ),
    MaxPooling2D((2,2)),
    Conv2D(32,(3,3) , activation = "relu" ),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(512, activation = 'relu'), # 64 neurons
    Dense(256, activation = 'relu'), # 64 neurons
    Dropout(0.2),
    Dense(1, activation = 'sigmoid'), # 2 output neuron

])


In [ ]:
model_3.summary()

In [ ]:
model_3.compile(optimizer="adam",loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
epochs=20
# history = model.fit_generator(
#         train_generator,
#         epochs=nb_epochs,
#         validation_data=validation_generator,
#         verbose=1,
#         steps_per_epoch=10//batch_size,
#         validation_steps=10//batch_size,
#         callbacks=cal_b)


history = model_3.fit(
  train_generator,
  validation_data=validation_generator,
  epochs=epochs,
  verbose=1,
  steps_per_epoch=20//batch_size,
  validation_steps=10//batch_size,

)

In [ ]:
image_datagen_test = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                         horizontal_flip=True)



image_generator_test = image_datagen_test.flow_from_directory(
    '/content/indian-document-forgery-detection/dataset/test',
    class_mode="binary",
    color_mode='rgb',
    target_size=(img_height,img_width),
    batch_size=batch_size,
    seed=2)




test_generator =image_generator_test

results = model_3.evaluate(test_generator)
print("Test Loss:", results[0])
print("Test Accuracy:", results[1])

In [ ]:
y_pred = model_3.predict(test_generator)
y_pred_classes = (y_pred > 0.5).astype(int)

y_true = test_generator.classes

plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix(y_true, y_pred_classes), annot=True, fmt='d', cmap='Blues',
            xticklabels=["Forged", "Real"],
            yticklabels=["Forged ", "Real"])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
filenames = test_generator.filenames
for i in range(len(filenames)):
    img_path = os.path.join(test_path, filenames[i])
    img = image.load_img(img_path, target_size=(256, 256))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = model_3.predict(img_array)
    predicted_class = int(np.round(prediction)[0])

    true_class = int(y_true[i])

    plt.imshow(img)
    plt.title(f"True Class: {true_class}, Predicted Class: {predicted_class}")
    plt.axis('off')
    plt.show()